In [2]:
import numpy as np
from numpy.lib import stride_tricks
import os
from PIL import Image
import scipy.io.wavfile as wav
import librosa  
import errno
import glob
import csv

In [3]:
# samplerate, samples = wav.read("300_AUDIO_8000.wav")

In [4]:
# samplerate

In [5]:
# len(samples)

In [6]:
inputDir = os.getcwd() + "/participantsAudio"

if(not os.path.exists(inputDir)):
    print("Input Directory Not Found")
else:
    listOfFile = os.listdir(inputDir)

listOfFile

['300_AUDIO.wav',
 '301_AUDIO.wav',
 '302_AUDIO.wav',
 '303_AUDIO.wav',
 '304_AUDIO.wav',
 '305_AUDIO.wav',
 '306_AUDIO.wav',
 '307_AUDIO.wav',
 '308_AUDIO.wav',
 '309_AUDIO.wav',
 '310_AUDIO.wav',
 '311_AUDIO.wav',
 '312_AUDIO.wav',
 '313_AUDIO.wav',
 '314_AUDIO.wav',
 '315_AUDIO.wav',
 '316_AUDIO.wav',
 '317_AUDIO.wav',
 '318_AUDIO.wav',
 '319_AUDIO.wav',
 '320_AUDIO.wav',
 '321_AUDIO.wav',
 '322_AUDIO.wav',
 '323_AUDIO.wav',
 '324_AUDIO.wav',
 '325_AUDIO.wav',
 '326_AUDIO.wav',
 '327_AUDIO.wav',
 '328_AUDIO.wav',
 '329_AUDIO.wav',
 '330_AUDIO.wav',
 '331_AUDIO.wav',
 '332_AUDIO.wav',
 '333_AUDIO.wav',
 '334_AUDIO.wav',
 '335_AUDIO.wav',
 '336_AUDIO.wav',
 '337_AUDIO.wav',
 '338_AUDIO.wav',
 '339_AUDIO.wav',
 '340_AUDIO.wav',
 '341_AUDIO.wav',
 '343_AUDIO.wav',
 '344_AUDIO.wav',
 '345_AUDIO.wav',
 '346_AUDIO.wav',
 '347_AUDIO.wav',
 '348_AUDIO.wav',
 '349_AUDIO.wav',
 '350_AUDIO.wav',
 '351_AUDIO.wav',
 '352_AUDIO.wav',
 '353_AUDIO.wav',
 '354_AUDIO.wav',
 '355_AUDIO.wav',
 '356_AUDI

In [7]:
outputDir = inputDir + "/output/10s"
if (not os.path.exists(outputDir)):
    try:
        os.makedirs(outputDir)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

listOfWavName = []
    
for item in listOfFile:
    if(item.endswith(".wav")):
        filename = item.split(".wav")[0]
        path = outputDir+"/"+filename
        listOfWavName.append(filename)
        if (not os.path.exists(path)):
            try:
                os.makedirs(path)
            except OSError as e:
                if e.errno != errno.EEXIST:
                    raise

In [61]:
def processFile(filename, file):
    samples, samplerate = librosa.load(filename, sr=8000)
    #INSERT LOOP FOR FRAMING 10s DATA
    totalSamples = len(samples)
    blockPerSegment = 80000
    cNumOfSegments = int(-(-totalSamples//blockPerSegment))
    fNumOfSegments = int(totalSamples//blockPerSegment)
    segments = []
    counter = 0

    for i in range(fNumOfSegments):
    #     segments.append(1)
        segments.append({(i)*blockPerSegment + (1 if i > 0 else 0), (i+1)*blockPerSegment})
    
    # segments
    for x, y in segments:
        if(x == 0):
            shortSamples = samples[x:y]
        else:
            shortSamples = samples[y:x]
        
        frameSize = 200
        hopsize = 80
        overlap = 120
        
        win = np.hanning(frameSize) #Forcuses on the peak (in the middle)
        s = np.append(np.zeros(overlap), shortSamples)
        # cols for windowing
        cols = np.ceil((len(s) - frameSize) / hopsize) + 1
        # zeros at end (thus samples can be fully covered by frames)
        s = np.append(s, np.zeros(overlap))
        
        frames = stride_tricks.as_strided(s, shape=(int(cols), frameSize),
                                          strides=(s.strides[0]*hopsize,
                                                   s.strides[0])).copy()
        frames *= win
        rfft = np.fft.rfft(frames)
        timebins, freqbins = np.shape(rfft)
        scale = np.linspace(0, 1, freqbins) ** 1
        scale *= (freqbins-1)/max(scale)
        scale = np.unique(np.round(scale))
        
        # create spectrogram with new freq bins
        newspec = np.complex128(np.zeros([timebins, len(scale)]))
        for i in range(0, len(scale)):
            if i == len(scale)-1:
                newspec[:, i] = np.sum(rfft[:, int(scale[i]):], axis=1)
            else:
                newspec[:, i] = np.sum(rfft[:, int(scale[i]):int(scale[i+1])], axis=1)
                
        # list center freq of bins
        allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./8000)[:freqbins+1])
        
        freqs = []
        for i in range(0, len(scale)):
            if i == len(scale)-1:
                freqs += [np.mean(allfreqs[int(scale[i]):])]
            else:
                freqs += [np.mean(allfreqs[int(scale[i]):int(scale[i+1])])]
                
        ims = 20.*np.log10(np.abs(newspec)/10e-6)  # amplitude to decibel
        
        ims = np.transpose(ims)
        ims = np.flipud(ims)  # weird - not sure why it needs flipping
        ims.dtype
#         image = Image.new()
        image = Image.fromarray(ims)
        image = image.convert('L')  # convert to grayscale
        if(x == 0):
            image.save("participantsAudio/output/10s/" + file + "/" + file + "("+ str(x) + "-"+ str(y) + ")"+".png")
        else:
            image.save("participantsAudio/output/10s/" + file + "/" + file + "("+ str(y) + "-"+ str(x) + ")"+".png")
        image.close()

    
for item in listOfWavName:
    filepath = "participantsAudio/" + item + ".wav"
    processFile(filepath, item)

C:\Users\hong ray\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: divide by zero encountered in log10


In [9]:
currentDir = os.getcwd()
datasetDir = currentDir + "/FilteredImages/"
# datasetDir = currentDir + "/FilteredImages2/" # NEW SINGLE IMAGES DIRECTORY
trainDir = os.path.join(datasetDir, "train")
testDir = os.path.join(datasetDir, "test")
validDir = os.path.join(datasetDir, "valid")
y_dataDir = os.path.join(datasetDir, "y_data")

In [10]:
def sortFirst(val):
    return val[0]

def getBinary(dataFile):
    listOfTraining = []
    trainingHeader = []
    with open(dataFile) as csvfile:
#     reader = csv.DictReader(csvfile)
        reader2 = csv.reader(csvfile)
        listOfTraining = list(reader2)
        trainingHeader = listOfTraining[0]
        listOfTraining.pop(0)
#         listOfTraining = listOfTraining.sort(key = sortFirst, reverse = False)
#         np.asarray(listofTraining, dtype=np.int32)
#         return np.asarray(listofTraining, dtype=np.int32)
    listOfTrainingBinary = []
    for item in listOfTraining:
        listOfTrainingBinary.append(item[1])
    return np.asarray(listOfTrainingBinary, dtype=np.int32)

In [11]:
y_train = []
y_trainDir = os.path.join(y_dataDir, 'train_split_Depression_AVEC2017-edited.csv')
# print(y_trainDir)
y_train = getBinary(y_trainDir)
y_testDir = os.path.join(y_dataDir, 'dev_split_Depression_AVEC2017.csv')
# print(y_testDir)
y_test = getBinary(y_testDir)
print(y_train)
print(y_test)

# Y_train = np.asarray(y_train, dtype=np.int32)

[0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1
 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0
 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0]


In [12]:
x_trainDir = trainDir
x_testDir = testDir

In [13]:
# testSet = []
# testFolders =[]
# # trainingSet = []
# trainingFolders = []
# x_train = []
# y_train = []

def getImagesDir(mainDirectory):
    ImgDict = {}
    ImgList = []
    
    for folder in os.listdir(mainDirectory):
        theDir = os.path.join(mainDirectory, folder)
        images = os.listdir(theDir)
        listOfImgDir = []
        for img in images:
#             listOfImgDir.append(Image.open(os.path.join(theDir, img)))
            listOfImgDir.append(os.path.join(theDir, img))
#         print(listOfImgDir)
#         print(images)
        folderName = folder.split('_')
        ImgDict[folderName[0]] = listOfImgDir
        ImgList.append(listOfImgDir)
    imgList = ImgList.sort(key = sortFirst, reverse = False)
    return ImgList
        
trainingImagesDir = getImagesDir(trainDir)
testImagesDir = getImagesDir(testDir)
# np.array(trainingImages).shape

In [14]:
def importImages(listOfImgDir):
    myFolder = []
    for folder in listOfImgDir:
        myImages = []
        for image in folder:
            myImages.append(np.array(Image.open(image)))
        myFolder.append(np.array(myImages))
    return myFolder


#First Array iterate through Folder, Second Array Iterate though Image in Folder
trainingImages = importImages(trainingImagesDir)
testImages = importImages(testImagesDir)

In [15]:
#Num of Img, Height, Width
print(len(trainingImages)) 

len(y_train)

107


107

In [16]:
trainingImagesNP = np.array(trainingImages)
testImagesNP = np.array(testImages)

In [17]:
# print(trainingImages.shape)
# print(trainingImages[0])
# print(trainingImagesNP[10].shape)
# print(y_train)

neg = []
y_neg = []
pos = []
y_pos = []

for x,y in zip(trainingImages, y_train):
    if (y == 0):
        for each in x:
            neg.append(each)
            y_neg.append(0)
    else:
        for each in x:
            pos.append(each)
            y_pos.append(1)

X = pos + neg
Y = y_pos + y_neg

In [50]:
neg_test = []
y_neg_test = []
pos_test = []
y_pos_test = []

for x,y in zip(testImages, y_train):
    if (y == 0):
        for each in x:
            neg_test.append(each)
            y_neg_test.append(0)
    else:
        for each in x:
            pos_test.append(each)
            y_pos_test.append(1)

X_test = pos_test + neg_test
Y_test = y_pos_test + y_neg_test


npX_test = np.array(X_test)
npY_test = np.array(Y_test)
newNPX_test = npX_test.reshape(npX_test.shape[0] * 10, 101, 100, 1)

In [51]:
vImagesDir = getImagesDir(validDir)
vImages = importImages(vImagesDir)
vImagesNP = np.array(vImages)

#TESTING DATA UNSEEN DATA
v = []
# y_neg_v = []
# pos_v = []
# y_pos_v = []

for x in vImages:
    for each in x:
        v.append(each)

np_v = np.array(v)
# npY_test = np.array(Y_test)
newNPX_v = np_v.reshape(np_v.shape[0] * 10, 101, 100, 1)

In [52]:
npX = np.array(X)
npY = np.array(Y)

print(npX.shape)
print(npY.shape)

(4591, 101, 1000)
(4591,)


In [67]:
newNPX = npX.reshape(npX.shape[0] * 10, 101, 100, 1)
# newNPX = np.array_split(newNPX, 10)

# newNPX = npX.reshape(npX.shape[0], 101, 1000, 1)

In [68]:
print(newNPX.shape)
print(newNPX_test.shape)
print(newNPX_v.shape)

(45910, 101, 100, 1)
(17400, 101, 100, 1)
(23810, 101, 100, 1)


In [69]:
newNPX = np.transpose(newNPX)
newNPX = np.flipud(newNPX) 

image = Image.fromarray(newNPX.astype('uint8'))
image = image.convert('L')  # convert to grayscale

if(x == 0):
    image.save("participantsAudio/output/10s/new/" + file + "/" + file + "("+ str(x) + "-"+ str(y) + ")"+".png")
else:
    image.save("participantsAudio/output/10s/new/" + file + "/" + file + "("+ str(y) + "-"+ str(x) + ")"+".png")
    image.close()

TypeError: Cannot handle this data type

In [ ]:
image = Image.fromarray(newNPX_test)
image = image.convert('L')  # convert to grayscale

if(x == 0):
    image.save("participantsAudio/output/10s/new/" + file + "/" + file + "("+ str(x) + "-"+ str(y) + ")"+".png")
else:
    image.save("participantsAudio/output/10s/new/" + file + "/" + file + "("+ str(y) + "-"+ str(x) + ")"+".png")
    image.close()

In [ ]:
image = Image.fromarray(newNPX_v)
image = image.convert('L')  # convert to grayscale

if(x == 0):
    image.save("participantsAudio/output/10s/new/" + file + "/" + file + "("+ str(x) + "-"+ str(y) + ")"+".png")
else:
    image.save("participantsAudio/output/10s/new/" + file + "/" + file + "("+ str(y) + "-"+ str(x) + ")"+".png")
    image.close()